In [1]:
import torch
import pyro
from pyro.optim import Adam, ClippedAdam
import congas as cg
from congas.models import MixtureCategorical, MixtureCategorical2, MixtureDirichlet
from pyro.infer import TraceMeanField_ELBO,TraceEnum_ELBO, TraceGraph_ELBO


In [2]:
data = cg.simulation_data

interface = cg.Interface()

In [3]:
data["norm_factor"] = torch.sum(data["data"],axis=0) / 10**3

In [4]:
torch.mean((data["data"] / data["norm_factor"]) / 2 )

tensor(15.6250)

In [11]:
interface.set_model(MixtureCategorical)
interface.set_optimizer(ClippedAdam)
interface.set_loss(TraceGraph_ELBO)
interface.initialize_model(data)
interface.set_model_params({"lr" : 0.01,'K': 2, "nb_size_init" : torch.ones(32) * 300,
                            "theta_scale" : torch.ones(32) * 1.5,
                            "theta_rate" : torch.ones(32) * 0.1,
                           "probs" : torch.tensor([0.2,0.6,0.2,0.05,0.05])})

In [12]:
ll = interface.run(steps = 500, MAP = True)



Running MixtureCategorical on 1000 cells wiht 32 segments for 500 steps

Done!


ELBO: 106373.072000000  : 100%|██████████| 500/500 [00:14<00:00, 35.58it/s]


In [13]:
import numpy as np

lr = interface.learned_parameters()
lr

Computing assignment probabilities


{'mixture_weights': array([0.6955218 , 0.30447823], dtype=float32),
 'NB_size': array([6001.4614, 7851.197 , 4123.702 , 6428.5693, 7146.3735, 4104.542 ,
        8943.439 , 3116.494 , 2701.1592, 8803.028 , 7564.94  , 4094.1895,
        6736.6274, 6328.3804, 1413.1351, 6184.788 , 2145.3516, 1346.6396,
        6850.7593, 9435.439 , 7367.5596, 2317.6804, 8416.429 , 6108.8096,
         590.4864, 8129.3286, 7085.8276, 8198.789 , 1302.6709, 8013.4697,
        2689.5356, 3456.9587], dtype=float32),
 'segment_factor': array([13.244415 , 14.819234 , 12.810789 , 14.411754 , 14.229758 ,
        13.376876 , 17.010983 , 10.64102  , 13.214214 , 16.186419 ,
        13.359442 , 15.655713 , 13.376876 , 13.201247 , 14.913476 ,
        10.181025 , 14.277906 , 10.090607 , 13.272441 , 25.42345  ,
        11.412315 , 14.1457815, 13.638575 , 13.828127 , 13.828127 ,
        13.833637 , 13.678261 , 14.23623  , 11.8657055, 12.4854   ,
         6.6277895, 11.452094 ], dtype=float32),
 'CNV_probabilities': array([

In [14]:

import numpy as np
np.argmax(lr["assignment_probs"], axis = 0)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [15]:
print(np.argmax(lr["CNV_probabilities"][0], axis = 1) + 1)
print(np.argmax(lr["CNV_probabilities"][1], axis = 1) + 1)


[2 3 1 2 1 2 4 1 2 4 1 2 2 2 2 2 2 2 2 4 2 3 2 1 1 2 2 2 1 2 1 1]
[2 2 1 2 1 2 4 2 2 3 1 1 2 3 3 1 2 1 2 4 1 2 3 1 1 3 3 2 2 2 1 2]


In [8]:
interface._model.likelihood(lr)

TypeError: matmul(): argument 'input' (position 1) must be Tensor, not numpy.ndarray